In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelcheckpointConfig:
    root_door: Path
    model_save: Path
    monitor: str
    save_best_only: bool
    save_weights_only: bool
    verbose: int
    patience: int
    restore_best_weights: bool
    factor: float
    min_lr: float
    existing_model: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml,create_directories

2024-09-21 18:18:16,402 - INFO - start of looging


In [7]:
class ConfigurationManagerone:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.data_root])

    def get_model_checkpoint_config(self) -> ModelcheckpointConfig:
        config = self.config.Model_Checkpoint
        create_directories([config.root_dir])

        modelcheckpoint_config = ModelcheckpointConfig(
            root_door = config.root_dir,
            model_save= config.model_save,
            monitor = self.params['Model_Checkpoint']['monitor'],
            save_best_only = self.params['Model_Checkpoint']['save_best_only'],
            save_weights_only = self.params['Model_Checkpoint']['save_weights_only'],
            verbose = self.params['Model_Checkpoint']['verbose'],
            patience = self.params['Early_stopping']['patience'],
            restore_best_weights = self.params['Early_stopping']['restore_best_weights'],
            factor= self.params['ReduceLROnPlateau']['factor'],
            min_lr= self.params['ReduceLROnPlateau']['min_lr'],
            existing_model=config.existing_model
        )

        return modelcheckpoint_config

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from src.components.data_aug import DataAugmentation
from src.entity import DataAugConfig
from src.config.configuration import ConfigurationManager

In [9]:
class Model_train_with_Callbacks:
    def __init__(self,config:ModelcheckpointConfig,data_config: DataAugConfig):
        self.config = config
        self.data_config = data_config

    @staticmethod
    def get_the_model_chceckpoint(self):
        model_check = ModelCheckpoint(
        filepath = self.config.model_save,
        monitor = self.config.monitor,
        verbose = self.config.verbose,
        save_best_only = self.config.save_best_only,
        save_weights_only=self.config.save_weights_only

    )

        early_stopping = EarlyStopping(
            monitor = self.config.monitor,
            patience =self.config.patience,
            verbose = self.config.verbose,
            restore_best_weights = self.config.restore_best_weights
        )

        reduce_lr = ReduceLROnPlateau(
            monitor=self.config.monitor,
            factor=self.config.factor,
            patience=self.config.patience,
            min_lr=self.config.min_lr,
    )
        return [model_check, early_stopping, reduce_lr]
    
    def model_training(self):
        model = load_model(self.config.existing_model)
        model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

        obj = DataAugmentation(config=self.data_config)  
        train_data, test_data, val_data = obj.augment_data()

        cb = self.get_the_model_chceckpoint(self)

        history = model.fit(train_data,epochs=1,validation_data=val_data,callbacks=cb)
        model.save(self.config.model_save)
        history.history
        return history, model

In [10]:
try:
    config = ConfigurationManagerone()
    mo_one = config.get_model_checkpoint_config()
    
    # Fetch the data augmentation configuration
    DataAugConfig_config = ConfigurationManager()  # Assuming this gets the data config
    c_2 = DataAugConfig_config.get_data_aug()  # Fetch the data augmentation config
    
    # Pass both model checkpoint config and data augmentation config
    mo_two = Model_train_with_Callbacks(config=mo_one, data_config=c_2)  
    mo_two.model_training()
except Exception as e:
    raise e


2024-09-21 18:18:19,145 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-21 18:18:19,147 - INFO - Yaml file: params.yaml loaded successfully
2024-09-21 18:18:19,148 - INFO - created directory at: data
2024-09-21 18:18:19,148 - INFO - created directory at: data/model_checkpoint
2024-09-21 18:18:19,150 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-21 18:18:19,152 - INFO - Yaml file: params.yaml loaded successfully
2024-09-21 18:18:19,152 - INFO - created directory at: data
Found 272 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 58 images belonging to 2 classes.


/Applications/anaconda3/envs/chkn/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/envs/chkn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4380 - loss: 1.6532
Epoch 1: accuracy improved from -inf to 0.53309, saving model to data/model_checkpoint/Chicken_fecal_with_callbacks.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 37s 4s/step - accuracy: 0.4475 - loss: 1.6221 - val_accuracy: 0.8793 - val_loss: 0.3036 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 1.
